# Ideia de similação

Na minha simulação de log, vou assumir que 60% dos usuários sabem o que estão atrás.

E que os 40% restantes atuam de acordo com o artigo do Thosten

1 0.4913494
2 0.1914648212226067
3 0.12687427912341406
4 0.10265282583621683
5 0.08765859284890426

In [2]:
import re
import pandas as pd
import nest_asyncio
from playwright.async_api import async_playwright, Page, expect
import asyncio
from random import randint
import numpy as np
import random


clicks = [0,]*10

df = pd.read_csv("queries_to_similate.csv")

# Allow nested event loops
nest_asyncio.apply()

async def run(playwright):

    n_clients = 10000
    queries = df['query'].to_list()

    print("Total queries:", len(queries))

    browser = await playwright.chromium.launch(headless=True)  # Launch the browser in non-headless mode
    page = await browser.new_page()


    for percent in range(n_clients):

        if percent%1000 == 0:
            print(f"{(percent/n_clients)*100}%")

        # 1. Access the address: http://localhost:3000/
        await page.goto('http://localhost:3000/')


        prob = 1 - np.random.uniform(0, 1) 
        
        if len(queries)==1 or prob > 0.4:

            query_index = randint(0, len(queries)-1)

            possible_animes = df[df['query']==queries[query_index]]['relevante_documents'].to_list()[0].split("|") 

            # anime_index = randint(0, len(possible_animes)-1)

            search_term = queries[query_index]

            random.shuffle(possible_animes)

            num_terms = random.randint(1, len(possible_animes))
            find_term = [anime.lower() for anime in possible_animes[:num_terms]]

            find_term = [ anime.lower() for anime in possible_animes]
    
            # 2. Find the input: //*[@id="hero"]/div/div/p/form/div/input
            input_locator = page.locator('//*[@id="hero"]/div/div/p/form/div/input')

            # 3. Type: "Shingeki no Kyojin"
            await input_locator.fill(search_term)

            # 4. Press the button: //*[@id="hero"]/div/div/p/form/button
            button_locator = page.locator('//*[@id="hero"]/div/div/p/form/button')
            await button_locator.click()

            # Wait for the results to load
            await page.wait_for_selector('//*[@id="root"]/div[2]/div/div')

            # 5. Get all child divs: //*[@id="root"]/div[2]/div
            child_divs_locator = page.locator('//*[@id="root"]/div[2]/div/div')

            # 6. For each child div, check if the text is "Shingeki no Kyojin"
            count = await child_divs_locator.count()
            
            try:
                for i in range(count-1):
                    text_locator = page.locator(f'//*[@id="root"]/div[2]/div/div[{i+1}]/button/div/div')
                    text = await text_locator.inner_text()
            
                    if any(term.lower() in text.lower() for term in find_term):
            
                        # 7. If found the term, press the button: //*[@id="root"]/div[2]/div/div[{i+1}]/button
                        button_to_click = page.locator(f'//*[@id="root"]/div[2]/div/div[{i+1}]/button')
                        await button_to_click.click()

                        clicks[i] += 1

                        await page.mouse.click(x=10, y=10)

            except:
                continue

        else:
            query_index = randint(0, len(queries)-1)

            search_term = queries[query_index]

            prob = np.random.uniform(0, 1) 

            if prob <= 0.08765859284890426:
                index_click = 5
            elif prob <= 0.10265282583621683:
                index_click = 4
            elif prob <= 0.12687427912341406:
                index_click = 3
            elif prob <= 0.1914648212226067:
                index_click = 2
            else:
                index_click = 1

            try:
                # 2. Find the input: //*[@id="hero"]/div/div/p/form/div/input
                input_locator = page.locator('//*[@id="hero"]/div/div/p/form/div/input')

                # 3. Type: "Shingeki no Kyojin"
                await input_locator.fill(search_term)

                # 4. Press the button: //*[@id="hero"]/div/div/p/form/button
                button_locator = page.locator('//*[@id="hero"]/div/div/p/form/button')
                await button_locator.click()

                # Wait for the results to load
                await page.wait_for_selector('//*[@id="root"]/div[2]/div/div')
            
                button_to_click = page.locator(f'//*[@id="root"]/div[2]/div/div[{index_click}]/button')
                await button_to_click.click()
                clicks[i] += 1
            except:
                continue

    await browser.close()

async def main():
    async with async_playwright() as playwright:
        await run(playwright)

# Run the main function
if __name__ == "__main__":
    asyncio.run(main())


Total queries: 223
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


In [3]:
print(clicks)

[882, 586, 518, 506, 255, 240, 434, 310, 332, 285]


Com o rank

[882, 586, 518, 506, 255, 240, 434, 310, 332, 285]

[1033, 827, 759, 733, 633, 650, 599, 626, 576, 544]